In [2]:
import os, sys
sys.path.append('..')
import common, sql
import pandas as pd

In [4]:
q='''select * from tool
where id not in (5);'''
tools=sql.execute(q)
tools

[{'id': 1, 'name': 'Maven Security Versions'},
 {'id': 2, 'name': 'Snyk'},
 {'id': 3, 'name': 'Github Dependabot'},
 {'id': 4, 'name': 'OWASP Dependency-Check'}]

In [5]:
for tool in tools:
    toolId = tool['id']
    toolName = tool['name']
    q='''select r.id, ifnull(sub.alerts,0) as alerts from repository r
        left join
        (select repositoryId, sum(mA.count)as alerts from mavenAlert mA
        join dependency d on mA.dependencyId = d.id
        where toolId=%s
        group by repositoryId) sub
        on r.id=sub.repositoryId
        where r.id != 44
        order by r.id;'''
    df=pd.DataFrame(sql.execute(q,(toolId,)))
    print(toolName, df.alerts.sum(), df.alerts.median())

Maven Security Versions 3197 58.0
Snyk 4968 62.0
Github Dependabot 136 0.0
OWASP Dependency-Check 12833 253.0


In [14]:
q='''select t.id, t.name, count(distinct p.id)
from mavenAlert mA
join dependency d on mA.dependencyId = d.id
join package p on d.packageId = p.id
join tool t on mA.toolId = t.id
group by t.id,t.name
order by t.id;'''
df=sql.pd_read_sql(q)
df

,id,name,count(distinct p.id)
0,1,Maven Security Versions,36
1,2,Snyk,96
2,3,Github Dependabot,20
3,4,OWASP Dependency-Check,354


In [6]:
for tool in tools:
    toolId = tool['id']
    toolName = tool['name']
    q='''select r.id, ifnull(sub.deps,0) as deps from repository r
        left join
        (select repositoryId, count(distinct packageId) as deps from mavenAlert mA
        join dependency d on mA.dependencyId = d.id
        where toolId=%s
        group by repositoryId) sub
        on r.id=sub.repositoryId
        where r.id != 44
        order by r.id;'''
    df=pd.DataFrame(sql.execute(q,(toolId,)))
    print(toolName, df.deps.median())

Maven Security Versions 12.0
Snyk 6.0
Github Dependabot 0.0
OWASP Dependency-Check 38.0


In [15]:
q='''select t.id, t.name, count(distinct p.artifact)
from mavenAlert mA
join dependency d on mA.dependencyId = d.id
join package p on d.packageId = p.id
join tool t on mA.toolId = t.id
group by t.id,t.name
order by t.id;'''
df=sql.pd_read_sql(q)
df

,id,name,count(distinct p.artifact)
0,1,Maven Security Versions,14
1,2,Snyk,46
2,3,Github Dependabot,11
3,4,OWASP Dependency-Check,181


In [7]:
for tool in tools:
    toolId = tool['id']
    toolName = tool['name']
    q='''select r.id, ifnull(sub.deps,0) as packages from repository r
        left join
        (select repositoryId, count(distinct p.artifact) as deps from mavenAlert mA
        join dependency d on mA.dependencyId = d.id
        join package p on d.packageId = p.id
        where toolId=%s
        group by repositoryId) sub
        on r.id=sub.repositoryId
        where r.id != 44
        order by r.id;'''
    df=pd.DataFrame(sql.execute(q,(toolId,)))
    print(toolName, df.packages.median())

Maven Security Versions 12.0
Snyk 6.0
Github Dependabot 0.0
OWASP Dependency-Check 37.0


In [18]:
q='''select t.id, t.name, count(distinct v.id)
from mavenAlert mA
join dependency d on mA.dependencyId = d.id
join package p on d.packageId = p.id
join tool t on mA.toolId = t.id
join vulnerability v on mA.vulnerabilityId = v.id
group by t.id,t.name
order by t.id;'''
df=sql.pd_read_sql(q)
df

,id,name,count(distinct v.id)
0,1,Maven Security Versions,36
1,2,Snyk,186
2,3,Github Dependabot,61
3,4,OWASP Dependency-Check,510


In [8]:
for tool in tools:
    toolId = tool['id']
    toolName = tool['name']
    q='''select r.id, ifnull(sub.vuln,0) as vuln from repository r
        left join
        (select repositoryId, count(distinct v.id) as vuln from mavenAlert mA
        join dependency d on mA.dependencyId = d.id
        join package p on d.packageId = p.id
        join vulnerability v on mA.vulnerabilityId = v.id
        where toolId=%s
        group by repositoryId) sub
        on r.id=sub.repositoryId
        where r.id != 44
        order by r.id;'''
    df=pd.DataFrame(sql.execute(q,(toolId,)))
    print(toolName, df.vuln.median())

Maven Security Versions 22.0
Snyk 22.0
Github Dependabot 0.0
OWASP Dependency-Check 119.0


In [22]:
q='''select t.id, t.name, count(distinct v.id)
from mavenAlert mA
join dependency d on mA.dependencyId = d.id
join package p on d.packageId = p.id
join tool t on mA.toolId = t.id
join vulnerability v on mA.vulnerabilityId = v.id
where v.cveId is not null and v.sourceId is null
group by t.id,t.name
order by t.id;'''
df=sql.pd_read_sql(q)
df

,id,name,count(distinct v.id)
0,1,Maven Security Versions,36
1,2,Snyk,175
2,3,Github Dependabot,61
3,4,OWASP Dependency-Check,286


In [23]:
q='select * from scanMinutes'
df=sql.pd_read_sql(q)
df

,toolId,maven,npm
0,1,2.46667,NaN
1,2,16.36670,1.03333
2,4,10.73330,4.43333


In [11]:
q='''select t.name as tool, repositoryId, count(distinct packageId) as c
from mavenAlert mA
join dependency d on mA.dependencyId = d.id
join derivedMavenDependencyTree dMDT on d.id = dMDT.dependencyId
join tool t on mA.toolId = t.id
group by toolId, repositoryId;'''
results = sql.pd_read_sql(q)
for tool, df in results.groupby('tool'):
    print(tool,df.c.median()) 

Maven Security Versions 12.0
Snyk 6.0


In [7]:
q='''select t.name as tool, repositoryId, count(distinct v.id) as c
from mavenAlert mA
join dependency d on mA.dependencyId = d.id
join derivedMavenDependencyTree dMDT on d.id = dMDT.dependencyId
join vulnerability v on mA.vulnerabilityId = v.id
join tool t on mA.toolId = t.id
group by toolId, repositoryId;'''
results = sql.pd_read_sql(q)
for tool, df in results.groupby('tool'):
    print(tool,df.c.median()) 

Maven Security Versions 22.0
Snyk 26.5


In [24]:
q='''select t.name as tool, scope, count(distinct d.packageId) as deps, count(distinct v.id) as vuln
from mavenAlert mA
join dependency d on mA.dependencyId=d.id
join vulnerability v on mA.vulnerabilityId=v.id
join derivedMavenDependencyTree dMDT on d.id = dMDT.dependencyId
join tool t on t.id=mA.toolId
group by toolId, scope;'''
sql.pd_read_sql(q)

,tool,scope,deps,vuln
0,Maven Security Versions,compile,6,9
1,Maven Security Versions,provided,35,35
2,Maven Security Versions,test,5,9
3,Snyk,compile,40,97
4,Snyk,provided,75,181
5,Snyk,runtime,1,2
6,Snyk,test,21,22
7,Github Dependabot,compile,11,60
8,Github Dependabot,provided,8,43
9,Github Dependabot,runtime,1,1


In [10]:
q='''select t.name as tool, scope, count(distinct d.packageId) as deps, count(distinct v.id) as vuln
from npmAlert nA
join dependency d on nA.dependencyId=d.id
join vulnerability v on nA.vulnerabilityId=v.id
join npmDependencyTree nDT on d.packageId = nDT.packageId
join tool t on t.id=nA.toolId
group by toolId, scope;'''
df=sql.pd_read_sql(q)
df

,tool,scope,deps,vuln
0,Snyk,dev,83,105
1,Snyk,prod,12,24
2,Github Dependabot,dev,8,9
3,Github Dependabot,prod,5,9
4,OWASP Dependency-Check,dev,167,168
5,OWASP Dependency-Check,prod,57,52
6,NPM Audit,dev,52,57
7,NPM Audit,prod,13,10
